In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import itertools
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

def xrange(x):
    return iter(range(x))

In [111]:
# Import data
def load_data():
    full_data = pd.read_csv("X.csv")
    train_y = pd.read_csv("ytr.csv")
    # Rename columns to something more interpretable
    columns = (["reflectance_" + str(i) for i in range(7)]
               + ["solar_" + str(i) for i in range(5)] + ["id"])
    full_data.columns = columns
    # Add y to the data frame
    split = 98000
    y_id_dict = train_y.set_index("Id")["y"].to_dict()
    full_data.loc[:(split-1), "y"] = full_data.loc[:(split-1), "id"].replace(y_id_dict)

    train, test = full_data[:split], full_data[split:]
    return (train, test)

#columns = (["id"] + ["reflectance_" + str(i) for i in range(7)]
#           + ["solar_" + str(i) for i in range(5)] + ["y"])
#full_data = pd.read_csv("MODIS.csv", header=None, names=columns)
#split = 98000
#train, test = full_data[:split].copy(), full_data[split:].copy()
train, test = load_data()

# Parameters
outliers_threshold = 0.01
n_threads = -1
random_seed = 22

In [112]:
cols_excl = ["id", "y"]
cols_orig = [c for c in train.columns if c not in cols_excl]

In [113]:
# Parameters
learning_rate = 0.01
training_epochs = 5
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 5 # 1st layer number of features
n_hidden_2 = 5 # 2nd layer number of features
n_input = 12
n_classes = 1

In [114]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None,])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [115]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y, pred))))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [116]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train)/batch_size)
        train = train.sample(frac=1).reset_index(drop=True)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = train[cols_orig][i*batch_size:(i+1)*batch_size]
            batch_y = train["y"][i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
            
        label_value = batch_y 
        estimate = p
        
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print ("[*]----------------------------")
            for i in xrange(3):
                print ("label value:", label_value.as_matrix()[i], \
                    ", estimated value:", estimate[i])
            print ("[*]============================")
    print("Optimization Finished!")

Epoch: 0001 cost= 2.050355705
[*]----------------------------
label value: -3.95918163533 , estimated value: [-3.20718098]
label value: -3.64219621174 , estimated value: [-3.5021596]
label value: -5.03276584846 , estimated value: [-3.28145456]
[*]============================
Epoch: 0002 cost= 1.341062059
[*]----------------------------
label value: -4.21934685635 , estimated value: [-3.01938629]
label value: -4.61361537938 , estimated value: [-3.0800519]
label value: -1.55640993982 , estimated value: [-3.13498878]
[*]============================
Epoch: 0003 cost= 1.328776683
[*]----------------------------
label value: -2.47110582177 , estimated value: [-3.57824111]
label value: -3.94590725969 , estimated value: [-3.38996625]
label value: -3.47801024827 , estimated value: [-3.40397]
[*]============================
Epoch: 0004 cost= 1.323489384
[*]----------------------------
label value: -0.635470050456 , estimated value: [-3.30888128]
label value: -4.22300668974 , estimated value: [-3